In [4]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import IntegerParameter, HyperparameterTuner, ContinuousParameter

# The processing job will start and SageMaker will handle the rest. You can monitor the job status in the SageMaker console.

# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = 'arn:aws:iam::863397112005:role/service-role/AmazonSageMaker-ExecutionRole-20231109T153131'

# Define the estimator
estimator = SKLearn(
    entry_point='sklearn-random-forest-hpo.py',
    instance_type='ml.c5.2xlarge',
    framework_version='0.23-1',
    role=role,
    sagemaker_session=sagemaker_session
)

# Define hyperparameter ranges
hyperparameter_ranges = {
    'n_estimators': IntegerParameter(100, 200),
    'max_depth': IntegerParameter(3, 10),
    'learning_rate': ContinuousParameter(0.01, 0.2),
    'min_child_weight': IntegerParameter(1, 10),
    'subsample': ContinuousParameter(0.5, 1),
    'colsample_bytree': ContinuousParameter(0.5, 1),
}

# Define the objective metric name and type
objective_metric_name = 'test:recall'
objective_type = 'Maximize'

# Define metric definitions
metrics_definitions = [
    {"Name": "test:recall", "Regex": "test:recall: ([0-9\\.]+)"}
]

# Initialize hyperparameter tuner
tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metrics_definitions,
    max_jobs=2,
    max_parallel_jobs=1,
    objective_type=objective_type
)

tuner.fit({'train': 's3://sagemaker-us-east-1-863397112005/fraud-pipeline-1700393590/jf5qiebe21g1/Processing/output/fraud-train/', 
           'test': 's3://sagemaker-us-east-1-863397112005/fraud-pipeline-1700393590/jf5qiebe21g1/Processing/output/fraud-test/'})


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/zenysisaccount/Library/Application Support/sagemaker/config.yaml
Using provided s3_resource


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.....................................................................................................................!
